In [19]:
import os
import collections
import json

import pandas as pd
import tqdm

import util

In [20]:
# workdir
workdir = r'd:/workdir/ztcata-lab'
src = os.path.join(workdir, 'dataset.csv')
ztcata_path = './ztcata/中图分类_full.xlsx'

# 读取数据
df = util.read_csv(src)
print('数据集元素个数：', len(df))
print(df.head(1))

df_zt = util.read_excel(ztcata_path)
print('中图分类总数：', len(df_zt))
print(df_zt.head(1))

数据集元素个数： 578523
  kjcg_id     cata1 cata2       keywords
0  282644  TQ177.37   NaN  磷石膏粉;建筑磷;生产工艺
中图分类总数： 20885
  index                    name
0     A  马克思主义、列宁主义、毛泽东思想、邓小平理论


In [21]:
# 基本的检查
def basic_check(df):
    # kjcg_id空值数量
    null_num = len(df[df['kjcg_id'].isna()])
    print('kjcg_id空值数量：', null_num)
    
    # cata1空值数量
    null_num = len(df[df['cata1'].isna()])
    print('cata1空值数量：', null_num)
    
    # cata2空值数量
    null_num = len(df[df['cata2'].isna()])
    print('cata2空值数量：', null_num)
    
    # keywords空值数量
    null_num = len(df[df['keywords'].isna()])
    print('keywords空值数量：', null_num)
    

basic_check(df)

kjcg_id空值数量： 0
cata1空值数量： 0
cata2空值数量： 452462
keywords空值数量： 0


In [22]:
# 出现的分类目录数量与手中ztcata表格的对比
cata_uniqs = df['cata1'].unique().tolist()
print('出现的分类总数：', len(cata_uniqs))  

# 中图分类总数22885
ztcatas = df_zt['index'].tolist()
print('zt分类总数：', len(ztcatas)) 

# 不包含在分类中的数量
unpresents = [i for i in tqdm.tqdm(cata_uniqs) if i not in ztcatas]
print('未出现的分类总数：', len(unpresents))

# 总结：我手中的中图分类非常之不全面

  2%|█▎                                                                          | 408/23080 [00:00<00:05, 4051.03it/s]

出现的分类总数： 23080
zt分类总数： 20885


100%|██████████████████████████████████████████████████████████████████████████| 23080/23080 [00:06<00:00, 3753.10it/s]

未出现的分类总数： 13334


In [24]:
# 分类号出现频次
counter = collections.Counter(df['cata1'].values.tolist())
print(counter.most_common(5))

# 保存中间文件，查看分布
middle_path = os.path.join(workdir, 'middle.xlsx')
dfm = pd.DataFrame(counter.most_common(), columns=['code', '成果数量'])
util.save_excel(middle_path, dfm)

[('TP319', 7278), ('R285.5', 3363), ('X703', 3218), ('TP391.41', 3136), ('S513', 2640)]


In [26]:
# 按code收集keyword
# 暂时只收集cata1的数据
r = dict()
for code, g in df.groupby(by='cata1'):
    inner = {'code':code, 'number':len(g), 'keywords':';'.join(g['keywords'].values.tolist())}
    r[code] = inner

# 保存
result_path = os.path.join(workdir, 'dataset_group_by_code.json')
with open(result_path, 'w', encoding='utf-8') as f:
    json.dump(r, f, ensure_ascii=False)

以下为测试：